In [2]:
# Don't look at this notebook through a peephole
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
from astroquery.simbad import Simbad
customSimbad = Simbad()
customSimbad.remove_votable_fields('coordinates')
customSimbad.add_votable_fields('ra(d;A;ICRS;;)', 'dec(d;D;ICRS;;)')
result_table = customSimbad.query_objects(["NGC869", "NGC884"])
result_table.keep_columns(['MAIN_ID', 'RA_d_A_ICRS__', 'DEC_d_D_ICRS__'])
result_table.rename_column('MAIN_ID', 'ID')
result_table.rename_column('RA_d_A_ICRS__', 'RA')
result_table.rename_column('DEC_d_D_ICRS__', 'DEC')
result_table

ID,RA,DEC
,deg,deg
object,float64,float64
NGC 869,34.74100000,57.13400000
NGC 884,35.58400000,57.14900000


In [7]:
import astropy.units as u
import numpy as np
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia

def query_gaia(field_size, simbad_result):    

    cluster_center = SkyCoord(ra=np.mean(simbad_result['RA']), dec=np.mean(simbad_result['DEC']), unit=(u.degree, u.degree) , frame='icrs')
    cluster_ngc869 = SkyCoord(ra=simbad_result[0]['RA'], dec=simbad_result[0]['DEC'], unit=(u.degree, u.degree) , frame='icrs')
    cluster_ngc884 = SkyCoord(ra=simbad_result[1]['RA'], dec=simbad_result[1]['DEC'], unit=(u.degree, u.degree) , frame='icrs')

    Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"

    fields_of_interest = f"""--
    -- Gaia's identifier used to join data
    source_id, 
    --
    -- Sky location/angle stuff
    --
    ra, dec,             -- Sky Location
    pmra, pmdec,         -- Sky Proper motion
    parallax,           -- parallax (not sure if needed, given bailer jones distances)
    astrometric_params_solved, -- Filtering to make sure we get at least the 5 sky params above (bias introduced here)
    -- Calculated separation to each cluster center and the median
    DISTANCE(ra, dec, {cluster_ngc869.ra.degree}, {cluster_ngc869.dec.degree}) as separation_ngc_869,
    DISTANCE(ra, dec, {cluster_ngc884.ra.degree}, {cluster_ngc884.dec.degree}) as separation_ngc_884,
    DISTANCE(ra, dec, {cluster_center.ra.degree}, {cluster_center.dec.degree}) as separation_center,
    --
    -- Distance
    --
    r_med_geo,        -- Distance according to Bailer-Jones analysis
    --
    -- Photometry/color
    --
    phot_g_mean_mag,  -- Gaia photometry
    bp_rp,
    --Reddening and extinction
    ebpminrp_gspphot,
    ag_gspphot,
    azero_gspphot, -- This is as close as Gaia gets to A_v
    distance_gspphot -- Distance from photometry, trying this to make HR diagram work
    """


    query = f"""
    SELECT 
    --TOP 10
    {fields_of_interest}
    FROM {Gaia.MAIN_GAIA_TABLE}
    JOIN external.gaiaedr3_distance as d USING (source_id) -- For Bailer-Jones Distances
    --JOIN gaiadr3.astrophysical_parameters as ap USING (source_id) -- For Extinction in all bands
    WHERE 
        1 = CONTAINS(
            POINT('ICRS', ra, dec),
            BOX('ICRS', {cluster_center.ra.degree}, {cluster_center.dec.degree}, {field_size.value}, {field_size.value})
        )
        AND phot_g_mean_mag IS NOT NULL
        AND bp_rp IS NOT NULL
        AND astrometric_params_solved >= 31
    """
    job = Gaia.launch_job_async(query, verbose = True)
    r = job.get_results()
    r.info()
    return r
    



In [8]:
%%time
# This cell takes about a minute to run

from astropy.table import Table
r_narrow = query_gaia(u.Quantity(2, u.deg), result_table)
r_narrow.write('../data/Cluster-narrow.hdf5', format='hdf5', overwrite=True, serialize_meta=True)

Launched query: '
    SELECT 
    --TOP 10
    --
    -- Gaia's identifier used to join data
    source_id, 
    --
    -- Sky location/angle stuff
    --
    ra, dec,             -- Sky Location
    pmra, pmdec,         -- Sky Proper motion
    parallax,           -- parallax (not sure if needed, given bailer jones distances)
    astrometric_params_solved, -- Filtering to make sure we get at least the 5 sky params above (bias introduced here)
    -- Calculated separation to each cluster center and the median
    DISTANCE(ra, dec, 34.741, 57.134) as separation_ngc_869,
    DISTANCE(ra, dec, 35.584, 57.149) as separation_ngc_884,
    DISTANCE(ra, dec, 35.1625, 57.1415) as separation_center,
    --
    -- Distance
    --
    r_med_geo,        -- Distance according to Bailer-Jones analysis
    --
    -- Photometry/color
    --
    phot_g_mean_mag,  -- Gaia photometry
    bp_rp,
    --Reddening and extinction
    ebpminrp_gspphot,
    ag_gspphot,
    azero_gspphot, -- This is as close as G

/home/mtauraso/astr511-hw2/.snakemake/conda/9f6bfdea3ec59e9e32ddf47f2c61ee75_/lib/python3.9/site-packages/astropy/io/misc/hdf5.py:283: UserWarning: table path was not set via the path= argument; using default path __astropy_table__
  warnings.warn(


In [9]:
%%time
# This cell takes about 10 minutes to run
# Shamelessly stolen bounds of 7.5 deg (Zhong, Chen 2019)
r_broad = query_gaia(u.Quantity(7.5, u.deg), result_table)
r_broad.write('../data/Cluster-broad.hdf5', format='hdf5', overwrite=True, serialize_meta=True)

# This takes a long time to run, but it's <200M of data and only takes a dozen seconds to compute on Gaia's side. 
# Virtually all the time is transfer.

Launched query: '
    SELECT 
    --TOP 10
    --
    -- Gaia's identifier used to join data
    source_id, 
    --
    -- Sky location/angle stuff
    --
    ra, dec,             -- Sky Location
    pmra, pmdec,         -- Sky Proper motion
    parallax,           -- parallax (not sure if needed, given bailer jones distances)
    astrometric_params_solved, -- Filtering to make sure we get at least the 5 sky params above (bias introduced here)
    -- Calculated separation to each cluster center and the median
    DISTANCE(ra, dec, 34.741, 57.134) as separation_ngc_869,
    DISTANCE(ra, dec, 35.584, 57.149) as separation_ngc_884,
    DISTANCE(ra, dec, 35.1625, 57.1415) as separation_center,
    --
    -- Distance
    --
    r_med_geo,        -- Distance according to Bailer-Jones analysis
    --
    -- Photometry/color
    --
    phot_g_mean_mag,  -- Gaia photometry
    bp_rp,
    --Reddening and extinction
    ebpminrp_gspphot,
    ag_gspphot,
    azero_gspphot, -- This is as close as G

/home/mtauraso/astr511-hw2/.snakemake/conda/9f6bfdea3ec59e9e32ddf47f2c61ee75_/lib/python3.9/site-packages/astropy/io/misc/hdf5.py:283: UserWarning: table path was not set via the path= argument; using default path __astropy_table__
  warnings.warn(


CPU times: user 1min 37s, sys: 1.53 s, total: 1min 39s
Wall time: 5min 31s
